In [48]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad

In [2]:
with open('matrix_list.txt', 'r') as file:
    content = file.read()

In [36]:
content_list = content.split("\n")
grouped_list = [content_list[i:i+3] for i in range(0, len(content_list), 3)]

In [43]:
samples_prefixes = [sample[0].split("_")[1] for sample in grouped_list[::2]]

In [44]:
samples_prefixes = [f'{sample}_CSF_' for sample in samples_prefixes] + [f'{sample}_PBMC_' for sample in samples_prefixes]

In [46]:
adatas = {}
for (barcode, feature, matrix), prefix in zip(grouped_list, samples_prefixes):
    adatas[prefix] = sc.read_10x_mtx(
        '../GSE194078/MTX',
        var_names='gene_symbols',
        gex_only=False, 
        cache=True,
        prefix=prefix)


In [52]:
ad.concat([adatas['CYG_CSF_'], adatas['CYG_PBMC_']]).obs

/Users/nicolapitzalis/anaconda3/envs/chl/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


""
AAACCCACAAGACCGA-1
AAACCCACACGGCACT-1
AAACCCATCAGAATAG-1
AAACGAAAGGTAAAGG-1
AAACGAACACAGCCAC-1
...
TTTGTTGCAAGTGCAG-1
TTTGTTGGTCAACCAT-1
TTTGTTGGTGTGATGG-1
TTTGTTGGTTGGCCTG-1


In [56]:
a = ad.concat([adatas['CYG_CSF_'], adatas['CYG_PBMC_']])

/Users/nicolapitzalis/anaconda3/envs/chl/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [58]:
a.obs_names_make_unique()

In [59]:
a.obs

""
AAACCCACAAGACCGA-1
AAACCCACACGGCACT-1
AAACCCATCAGAATAG-1
AAACGAAAGGTAAAGG-1
AAACGAACACAGCCAC-1
...
TTTGTTGCAAGTGCAG-1
TTTGTTGGTCAACCAT-1
TTTGTTGGTGTGATGG-1
TTTGTTGGTTGGCCTG-1
